In [53]:
from transformers import AutoModel, AutoTokenizer
import torch
from tqdm import tqdm
from pickle import load, dump

In [61]:
model = AutoModel.from_pretrained("bert-large-uncased").cuda()
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [66]:
with open("metrics/blip2_cs10k.pkl", "rb") as f:
    blip2_cs10k = load(f)

with open("metrics/blip2_cs10k_np.pkl", "rb") as f:
    blip2_cs10k_np = load(f)

with open("metrics/blip2_cs10k_ft.pkl", "rb") as f:
    blip2_cs10k_ft = load(f)

with open("metrics/blip2_cs10k.pkl", "rb") as f:
    blip2_cs10k = load(f)

with open("metrics/blip2_cs10k_np.pkl", "rb") as f:
    blip2_cs10k_np = load(f)

with open("metrics/blip2_cs10k_ft.pkl", "rb") as f:
    blip2_cs10k_ft = load(f)

def calculate_cosine_similarity(generated, model, tokenizer, cos):
    inp = tokenizer(generated, return_tensors="pt", padding=True,truncation=True).to("cuda")
    with torch.no_grad():
        out = model(**inp).last_hidden_state[:, 0, :]
    return cos(out[0].unsqueeze(0), out[1].unsqueeze(0)).cpu().numpy()
    


In [79]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

cosines = []
model_metrics = blip2_cs10k_ft
data = model_metrics['outputs']

for out in tqdm(data):
    cosines.append(calculate_cosine_similarity(out, model, tokenizer, cos))

with open('metrics/blip2_cs10k_ft_cosine.pkl', 'wb') as f:
    dump(cosines, f)



100%|██████████| 3049/3049 [01:37<00:00, 31.13it/s]


In [76]:
with open('metrics/blip2_cs10k_cosine.pkl', 'rb') as f:
    blip2_cs10k_cosine = load(f)

with open('metrics/blip2_cs10k_np_cosine.pkl', 'rb') as f:
    blip2_cs10k_np_cosine = load(f)

with open('metrics/blip2_cs10k_ft_cosine.pkl', 'rb') as f:
    blip2_cs10k_ft_cosine = load(f)

print(sum(blip2_cs10k_cosine)/len(blip2_cs10k_cosine))

print(sum(blip2_cs10k_np_cosine)/len(blip2_cs10k_np_cosine))

print(sum(blip2_cs10k_ft_cosine)/len(blip2_cs10k_ft_cosine))

[0.55020344]
[0.7162362]
[0.8192205]


In [80]:
with open('metrics/blip2_cs10k_cosine.pkl', 'rb') as f:
    blip2_cs10k_cosine = load(f)

with open('metrics/blip2_cs10k_np_cosine.pkl', 'rb') as f:
    blip2_cs10k_np_cosine = load(f)

with open('metrics/blip2_cs10k_ft_cosine.pkl', 'rb') as f:
    blip2_cs10k_ft_cosine = load(f)

print(sum(blip2_cs10k_cosine)/len(blip2_cs10k_cosine))

print(sum(blip2_cs10k_np_cosine)/len(blip2_cs10k_np_cosine))

print(sum(blip2_cs10k_ft_cosine)/len(blip2_cs10k_ft_cosine))


[0.52231216]
[0.72126746]
[0.8195744]
